In [0]:
import ftplib, os, socket
from ipyparallel import Client
from os import listdir as ls
from os import path as op
import shutil
import gzip
import pandas as pd
from collections import OrderedDict

# get the fastq files

In [0]:
DIR = '/home/lindb/wbp/concatenated'
files = [op.join(DIR,f) for f in ls(DIR) if f.endswith('compiled.fastq')]
len(files)

In [0]:
sorted(files)

# get engines

In [0]:
rc = Client(profile="xmn")
dview = rc[:]
lview = rc.load_balanced_view()
#small = rc.load_balanced_view(targets=range(0,10))
len(lview)#, len(small)

In [0]:
with dview.sync_imports():
    import os
    from os import path as op
    from os import listdir as ls
    import ftplib
    import socket
    import gzip
    import shutil

In [0]:
hosts = dview.apply(socket.gethostname)

In [0]:
host_dict = {}
for i, host in enumerate(hosts.get()):
    if not host in host_dict:
        host_dict[host] = []
    host_dict[host].append(i)
host_dict

In [0]:
single_host = rc.load_balanced_view(targets=[v[0] for k,v in host_dict.items()])

In [0]:
len(single_host)

# copy original files new a new folder

In [0]:
def cp(f):
    from os import path as op
    from os import listdir as ls
    import os
    import shutil
    DIR = op.dirname(f)
    gDIR = op.join(DIR,'sra')
    dst = op.join(gDIR, op.basename(f))
    shutil.copy(f,dst)
    return dst

In [0]:
dview['upload_file'] = download_file
dview['connect'] = connect
dview['cp'] = cp

In [0]:
jobs = []
for f in files:
    jobs.append(lview.apply_async(cp,f))

In [0]:
count = 0
for j in jobs:
    if j.ready():
        count += 1
count

In [0]:
DIR = '/home/lindb/wbp/concatenated/sra/'
gzfiles = [op.join(DIR,F) for F in ls(DIR) if F.endswith('fastq')]
len(gzfiles)

# gzip the copied files

In [0]:
def gzip_file(f):
    from os import path as op
    from os import listdir as ls
    import os
    DIR = op.dirname(f)
    dst = op.join(DIR,"%s.gz" % op.basename(f))
    !gzip -c $f > $dst
    return dst
dview['gzip_file'] = gzip_file

In [0]:
#gzip the files
jobs = []
for f in gzfiles:
    jobs.append(lview.apply_async(gzip_file,f))

In [0]:
count = 0
for j in jobs:
    if j.ready():
        count += 1
count

In [0]:
count = 0
for j in jobs:
    if j.ready():
        count +=1
count

In [0]:
DIR = '/home/lindb/wbp/concatenated/sra/'
gzfiles = [op.join(DIR,F) for F in ls(DIR) if F.endswith('gz')]
len(gzfiles)

# ftp upload the files

In [0]:
def connect():
    ftp = ftplib.FTP('ftp-private.ncbi.nlm.nih.gov','subftp','w4pYB9VQ')
    ftp.set_pasv(True)
    return ftp

def upload_file(f):
    print "uploading on %s on %s" % (f,socket.gethostname())
    from os import path as op
    import os
    ftp = connect()
    ftp.cwd('uploads/lindb@mymail.vcu.edu_S7FPFoO2/new_folder')
    filE = open(f,'rb')
    ftp.storbinary("STOR %s" % op.basename(f),filE)
    filE.close()
    ftp.quit()
    return op.basename(f)

dview['upload_file'] = upload_file
dview['connect'] = connect

In [0]:
DIR = '/home/lindb/wbp/concatenated/sra/'
gzfiles = [op.join(DIR,f) for f in ls(DIR) if f.endswith('gz')]
len(gzfiles)

In [0]:
jobs = []
for f in gzfiles:
    jobs.append(lview.apply_async(upload_file,f))

In [0]:
count= 0
for j in jobs:
    if j.ready():
        count += 1
count

# prepare sra docs

In [0]:
bioproject_id = '' #PRJNA377565
biosample_accession = 'SAMN06462359'
#library_ID
title = 'Local adaptation of Pinus albicaulis in the Lake Tahoe Basin'
library_strategy = 'OTHER'
library_source = 'GENOMIC'
library_selection = 'Reduced Representation'
library_layout = 'single'
platform = 'ILLUMINA'
instrument_model = 'Illumina HiSeq 2500'
design_description = 'OTHER'
filetype = 'fastq'
#filename
filename2 = ''
filename3 = ''
assembly = ''

In [0]:
DIR = '/home/lindb/wbp/concatenated/sra/'
gzfiles = sorted([op.join(DIR,f) for f in ls(DIR) if f.endswith('gz')])
data = []
for g in gzfiles:
    library_ID = op.basename(g).split(".")[0]
    filename = op.join('uploads/lindb@mymail.vcu.edu_S7FPFoO2/new_folder',op.basename(g))
    Dict = OrderedDict()
    Dict['bioproject_accession']=bioproject_id
    Dict['biosample_accession']=biosample_accession
    Dict['library_ID']=library_ID
    Dict['title']=title
    Dict['library_strategy']= library_strategy
    Dict['library_source'] = library_source
    Dict['library_selection'] = library_selection
    Dict['library_layout'] = library_layout
    Dict['platform'] = platform
    Dict['instrument_model']=instrument_model
    Dict['design_description']=design_description
    Dict['filetype']=filetype
    Dict['filename']=filename
    Dict['filename2']=filename2
    Dict['filename3']=filename3
    Dict['assembly']=assembly
    data.append(Dict)
df = pd.DataFrame(data,columns=data[0].keys())

In [0]:
df.head()

In [0]:
df.loc[0,'filename']

In [0]:
df.to_csv('/home/lindb/wbp/sra_metadata.tsv',header=True,index=False,sep='\t')